In [1]:
import pandas as pd
import numpy as np
import requests as req
import xarray as xr


## here are my notes
    - list
    - are great

In [3]:
!ls

Citymapper_Mobility_Index_20200501.csv
Citymapper_Mobility_Index_20200527.csv
Citymapper_Mobility_Index_20200528.csv
infoplease.html
OMI-Aura_L3-OMNO2d_2019m1101_v003-2020m0428t115333.he5.ascii
OMI-Aura_L3-OMNO2d_2019m1101_v003-2020m0428t115333.he5.nc4
OMI-Aura_L3-OMNO2d_2020m0509_v003-2020m0511t122736.he5.nc4
OMI-Aura_L3-OMNO2d_2020m0510_v003-2020m0511t173912.he5.ascii
OMI-Aura_L3-OMNO2d_2020m0510_v003-2020m0511t173912.he5.nc4
sa_chl_get_city_activity.ipynb
subset_OMNO2d_003_20200528_194433.txt
subset_OMNO2d_003_20200529_073831_cdef.txt
xcube


In [2]:
geo_dist_factor=2 #i.e 2 degrees of lat/long a square of approx 111*111km
'''
seconds are 30.916m
Each degree of Lats is 111km,each min (n-after decimal point 1.85km)
Each degree of lots is 111km,each min (n-after decimal point 1.85km)

'''

In [36]:
ds = xr.open_dataset('OMI-Aura_L3-OMNO2d_2019m1101_v003-2020m0428t115333.he5.nc4')
df = ds.to_dataframe()

In [4]:

ds = xr.open_dataset('OMI-Aura_L3-OMNO2d_2020m0509_v003-2020m0511t122736.he5.nc4')
df3 = ds.to_dataframe()

In [2]:
#a=req.get("https://www.infoplease.com/world/world-geography/major-cities-latitude-longitude-and-corresponding-time-zones").content
#open("infoplease.html","wb+").write(a)
d=pd.read_html("infoplease.html")[0]

In [3]:
d

Unnamed: 0_level_0 Latitude       Longitude       Unnamed: 5_level_0
                        City        °     '         °     '               Time
0         Aberdeen, Scotland       57   9 N         2   9 W          5:00 p.m.
1        Adelaide, Australia       34  55 S       138  36 E         2:30 a.m.1
2           Algiers, Algeria       36  50 N         3   0 E          6:00 p.m.
3     Amsterdam, Netherlands       52  22 N         4  53 E          6:00 p.m.
4             Ankara, Turkey       39  55 N        32  55 E          7:00 p.m.
..                       ...      ...   ...       ...   ...                ...
115          Vienna, Austria       48  14 N        16  20 E          6:00 p.m.
116      Vladivostok, Russia       43  10 N       132   0 E         3:00 a.m.1
117           Warsaw, Poland       52  14 N        21   0 E          6:00 p.m.
118  Wellington, New Zealand       41  17 S       174  47 E         5:00 a.m.1
119      Zürich, Switzerland       47  21 N         8  31 E          6:00 p.m.

[120 rows x 6 columns]

In [6]:
d

Unnamed: 0_level_0 Latitude       Longitude       Unnamed: 5_level_0
                        City        °     '         °     '               Time
0         Aberdeen, Scotland       57   9 N         2   9 W          5:00 p.m.
1        Adelaide, Australia       34  55 S       138  36 E         2:30 a.m.1
2           Algiers, Algeria       36  50 N         3   0 E          6:00 p.m.
3     Amsterdam, Netherlands       52  22 N         4  53 E          6:00 p.m.
4             Ankara, Turkey       39  55 N        32  55 E          7:00 p.m.
..                       ...      ...   ...       ...   ...                ...
115          Vienna, Austria       48  14 N        16  20 E          6:00 p.m.
116      Vladivostok, Russia       43  10 N       132   0 E         3:00 a.m.1
117           Warsaw, Poland       52  14 N        21   0 E          6:00 p.m.
118  Wellington, New Zealand       41  17 S       174  47 E         5:00 a.m.1
119      Zürich, Switzerland       47  21 N         8  31 E          6:00 p.m.

[120 rows x 6 columns]

In [100]:
countries_dict={}
for i in np.array(d):
    lat=i[1]+(int(i[2].split(" ")[0])*.10)
    lot=i[3]+(int(i[4].split(" ")[0])*.10)
    country_name=i[0].split(" ")[1]
    try: 
        countries_dict.update({country_name: countries_dict[country_name]+[(lat,lot)] })
    except KeyError:
        countries_dict.update({country_name:[(lat,lot)] })
print("Cities",sum([len(i) for i in countries_dict.keys()]))

Cities 514


In [98]:
def get_mean_ColumnAmountNO2(countries_dict):
    a=[]     
    c_dict={}
    idxes_file=np.array([[i[0],i[1]] for i in np.array(df3["ColumnAmountNO2"].index)])
    for c_name in countries_dict.keys():
        for c_idx in range(len(countries_dict[c_name])): # some contries have more than one city registred 
            x,y=countries_dict[c_name][c_idx]
            idx_of_xny=np.where (  # numpy black magic search for x,y pairs that are within geo_dist_factor of the x,y pairs
                (np.abs(idxes_file[:,0]-x)<=geo_dist_factor)
                & 
                (np.abs(idxes_file[:,1]-y)<=geo_dist_factor)
            )[0]
            
            for i in idx_of_xny:
                no2_amount=df3["ColumnAmountNO2"][i]
                if not (not no2_amount>0 and  not no2_amount<0): #basically eval if NaN
                    a.append(no2_amount/1000000000000000) # since all values seem to be scaled up ask Pandas idk?
            
        c_dict.update({c_name:np.mean(a)})
        a=[]
                        
    return c_dict
import time 
t=time.time()
a=get_mean_ColumnAmountNO2(countries_dict)
print(time.time()-t)

7.679959297180176


In [99]:
a

{'Scotland': 4.912165261541377,
 'Australia': 3.1907250890503365,
 'Algeria': 3.4680952997214143,
 'Netherlands': 5.901560126111745,
 'Turkey': 4.314634468196353,
 'Paraguay': 3.0197413939197704,
 'Greece': 4.8234467732103,
 'New': 3.636422127346294,
 'Thailand': 4.2493370875904,
 'Spain': -1.5658999283711998,
 'China': 6.24120017977344,
 'Brazil': 2.640353505971967,
 'Northern': 4.882264700747776,
 'Serbia': 4.208700291118173,
 'Germany': 5.522105791676416,
 'England': 7.7081241127485445,
 'Colombia': 2.25383345291264,
 'India': 3.6635839054323855,
 'France': 7.168980678787779,
 'Belgium': 4.859471900704768,
 'Romania': nan,
 'Hungary': 5.072586243225682,
 'Aires,': 4.314524036964122,
 'Egypt': nan,
 'Town,': 3.97206583508992,
 'Venezuela': 2.483471666642944,
 'French': 2.245879847452672,
 'Mexico': 3.05163509170176,
 'Denmark': 5.171802286850047,
 'Argentina': 4.006242637071428,
 'Senegal': 2.530155442470912,
 'Djibouti': 3.186877087333308,
 'Ireland': 5.3987035284766725,
 'South': 3

In [97]:
df3["ColumnAmountNO2"][330221]/1000000000000000

2.443100609839104

In [103]:
df3["ColumnAmountNO2"]

lat         lon        
-89.875000  -179.875015   NaN
            -179.624985   NaN
            -179.375015   NaN
            -179.124985   NaN
            -178.875015   NaN
                           ..
 89.875008   178.875015   NaN
             179.124985   NaN
             179.375015   NaN
             179.624985   NaN
             179.875015   NaN
Name: ColumnAmountNO2, Length: 1036800, dtype: float32

In [11]:
for i in df3["ColumnAmountNO2"].index:
    print(i)
    

(-89.875, -179.87501525878906)
(-89.875, -179.62498474121094)
(-89.875, -179.37501525878906)
(-89.875, -179.12498474121094)
(-89.875, -178.87501525878906)
(-89.875, -178.62498474121094)
(-89.875, -178.37501525878906)
(-89.875, -178.12498474121094)
(-89.875, -177.87501525878906)
(-89.875, -177.62498474121094)
(-89.875, -177.37501525878906)
(-89.875, -177.12498474121094)
(-89.875, -176.87501525878906)
(-89.875, -176.62498474121094)
(-89.875, -176.37501525878906)
(-89.875, -176.12498474121094)
(-89.875, -175.87501525878906)
(-89.875, -175.62498474121094)
(-89.875, -175.37501525878906)
(-89.875, -175.12498474121094)
(-89.875, -174.87501525878906)
(-89.875, -174.62498474121094)
(-89.875, -174.37501525878906)
(-89.875, -174.12498474121094)
(-89.875, -173.87501525878906)
(-89.875, -173.62498474121094)
(-89.875, -173.37501525878906)
(-89.875, -173.12498474121094)
(-89.875, -172.87501525878906)
(-89.875, -172.62498474121094)
(-89.875, -172.37501525878906)
(-89.875, -172.12498474121094)
(-89.875

(-89.875, 138.37501525878906)
(-89.875, 138.62498474121094)
(-89.875, 138.87501525878906)
(-89.875, 139.12498474121094)
(-89.875, 139.37501525878906)
(-89.875, 139.62498474121094)
(-89.875, 139.87501525878906)
(-89.875, 140.12498474121094)
(-89.875, 140.37501525878906)
(-89.875, 140.62498474121094)
(-89.875, 140.87501525878906)
(-89.875, 141.12498474121094)
(-89.875, 141.37501525878906)
(-89.875, 141.62498474121094)
(-89.875, 141.87501525878906)
(-89.875, 142.12498474121094)
(-89.875, 142.37501525878906)
(-89.875, 142.62498474121094)
(-89.875, 142.87501525878906)
(-89.875, 143.12498474121094)
(-89.875, 143.37501525878906)
(-89.875, 143.62498474121094)
(-89.875, 143.87501525878906)
(-89.875, 144.12498474121094)
(-89.875, 144.37501525878906)
(-89.875, 144.62498474121094)
(-89.875, 144.87501525878906)
(-89.875, 145.12498474121094)
(-89.875, 145.37501525878906)
(-89.875, 145.62498474121094)
(-89.875, 145.87501525878906)
(-89.875, 146.12498474121094)
(-89.875, 146.37501525878906)
(-89.875, 

(-89.625, 44.87500762939453)
(-89.625, 45.12499237060547)
(-89.625, 45.37500762939453)
(-89.625, 45.62499237060547)
(-89.625, 45.87500762939453)
(-89.625, 46.12499237060547)
(-89.625, 46.37500762939453)
(-89.625, 46.62499237060547)
(-89.625, 46.87500762939453)
(-89.625, 47.12499237060547)
(-89.625, 47.37500762939453)
(-89.625, 47.62499237060547)
(-89.625, 47.87500762939453)
(-89.625, 48.12499237060547)
(-89.625, 48.37500762939453)
(-89.625, 48.62499237060547)
(-89.625, 48.87500762939453)
(-89.625, 49.12499237060547)
(-89.625, 49.37500762939453)
(-89.625, 49.62499237060547)
(-89.625, 49.87500762939453)
(-89.625, 50.12499237060547)
(-89.625, 50.37500762939453)
(-89.625, 50.62499237060547)
(-89.625, 50.87500762939453)
(-89.625, 51.12499237060547)
(-89.625, 51.37500762939453)
(-89.625, 51.62499237060547)
(-89.625, 51.87500762939453)
(-89.625, 52.12499237060547)
(-89.625, 52.37500762939453)
(-89.625, 52.62499237060547)
(-89.625, 52.87500762939453)
(-89.625, 53.12499237060547)
(-89.625, 53.3

(-89.375, -46.875)
(-89.375, -46.625)
(-89.375, -46.375)
(-89.375, -46.125)
(-89.375, -45.875)
(-89.375, -45.62499237060547)
(-89.375, -45.37500762939453)
(-89.375, -45.12499237060547)
(-89.375, -44.87500762939453)
(-89.375, -44.62499237060547)
(-89.375, -44.37500762939453)
(-89.375, -44.12499237060547)
(-89.375, -43.87500762939453)
(-89.375, -43.62499237060547)
(-89.375, -43.37500762939453)
(-89.375, -43.12499237060547)
(-89.375, -42.87500762939453)
(-89.375, -42.62499237060547)
(-89.375, -42.37500762939453)
(-89.375, -42.12499237060547)
(-89.375, -41.87500762939453)
(-89.375, -41.62499237060547)
(-89.375, -41.37500762939453)
(-89.375, -41.12499237060547)
(-89.375, -40.87500762939453)
(-89.375, -40.62499237060547)
(-89.375, -40.37500762939453)
(-89.375, -40.12499237060547)
(-89.375, -39.87500762939453)
(-89.375, -39.62499237060547)
(-89.375, -39.37500762939453)
(-89.375, -39.12499237060547)
(-89.375, -38.87500762939453)
(-89.375, -38.62499237060547)
(-89.375, -38.37500762939453)
(-89.

(-89.125, -129.125)
(-89.125, -128.875)
(-89.125, -128.625)
(-89.125, -128.375)
(-89.125, -128.125)
(-89.125, -127.875)
(-89.125, -127.625)
(-89.125, -127.375)
(-89.125, -127.125)
(-89.125, -126.875)
(-89.125, -126.625)
(-89.125, -126.375)
(-89.125, -126.125)
(-89.125, -125.875)
(-89.125, -125.625)
(-89.125, -125.375)
(-89.125, -125.125)
(-89.125, -124.875)
(-89.125, -124.625)
(-89.125, -124.375)
(-89.125, -124.125)
(-89.125, -123.875)
(-89.125, -123.625)
(-89.125, -123.375)
(-89.125, -123.125)
(-89.125, -122.875)
(-89.125, -122.625)
(-89.125, -122.375)
(-89.125, -122.125)
(-89.125, -121.875)
(-89.125, -121.625)
(-89.125, -121.375)
(-89.125, -121.125)
(-89.125, -120.875)
(-89.125, -120.625)
(-89.125, -120.375)
(-89.125, -120.125)
(-89.125, -119.875)
(-89.125, -119.625)
(-89.125, -119.375)
(-89.125, -119.125)
(-89.125, -118.875)
(-89.125, -118.625)
(-89.125, -118.375)
(-89.125, -118.125)
(-89.125, -117.875)
(-89.125, -117.625)
(-89.125, -117.375)
(-89.125, -117.125)
(-89.125, -116.875)


...

(-85.625, -71.125)
(-85.625, -70.875)
(-85.625, -70.625)
(-85.625, -70.375)
(-85.625, -70.125)
(-85.625, -69.875)
(-85.625, -69.625)
(-85.625, -69.375)
(-85.625, -69.125)
(-85.625, -68.875)
(-85.625, -68.625)
(-85.625, -68.375)
(-85.625, -68.125)
(-85.625, -67.875)
(-85.625, -67.625)
(-85.625, -67.375)
(-85.625, -67.125)
(-85.625, -66.875)
(-85.625, -66.625)
(-85.625, -66.375)
(-85.625, -66.125)
(-85.625, -65.875)
(-85.625, -65.625)
(-85.625, -65.375)
(-85.625, -65.125)
(-85.625, -64.875)
(-85.625, -64.625)
(-85.625, -64.375)
(-85.625, -64.125)
(-85.625, -63.875)
(-85.625, -63.625)
(-85.625, -63.375)
(-85.625, -63.125)
(-85.625, -62.875)
(-85.625, -62.625)
(-85.625, -62.375)
(-85.625, -62.125)
(-85.625, -61.875)
(-85.625, -61.625)
(-85.625, -61.375)
(-85.625, -61.125)
(-85.625, -60.875)
(-85.625, -60.625)
(-85.625, -60.375)
(-85.625, -60.125)
(-85.625, -59.875)
(-85.625, -59.625)
(-85.625, -59.375)
(-85.625, -59.125)
(-85.625, -58.875)
(-85.625, -58.625)
(-85.625, -58.375)
(-85.625, -5

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




(-85.375, -123.625)
(-85.375, -123.375)
(-85.375, -123.125)
(-85.375, -122.875)
(-85.375, -122.625)
(-85.375, -122.375)
(-85.375, -122.125)
(-85.375, -121.875)
(-85.375, -121.625)
(-85.375, -121.375)
(-85.375, -121.125)
(-85.375, -120.875)
(-85.375, -120.625)
(-85.375, -120.375)
(-85.375, -120.125)
(-85.375, -119.875)
(-85.375, -119.625)
(-85.375, -119.375)
(-85.375, -119.125)
(-85.375, -118.875)
(-85.375, -118.625)
(-85.375, -118.375)
(-85.375, -118.125)
(-85.375, -117.875)
(-85.375, -117.625)
(-85.375, -117.375)
(-85.375, -117.125)
(-85.375, -116.875)
(-85.375, -116.625)
(-85.375, -116.375)
(-85.375, -116.125)
(-85.375, -115.875)
(-85.375, -115.625)
(-85.375, -115.375)
(-85.375, -115.125)
(-85.375, -114.875)
(-85.375, -114.625)
(-85.375, -114.375)
(-85.375, -114.125)
(-85.375, -113.875)
(-85.375, -113.625)
(-85.375, -113.375)
(-85.375, -113.125)
(-85.375, -112.875)
(-85.375, -112.625)
(-85.375, -112.375)
(-85.375, -112.125)
(-85.375, -111.875)
(-85.375, -111.625)
(-85.375, -111.375)

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-94ca240b6ccb>", line 2, in <module>
    print(i)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 402, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 205, in schedule
    self._event_pipe.send(b'')
  File "/usr/local/lib/python3.6/dist-packages/zmq/sugar/socket.py", line 400, in send
    return super(Socket, self).send(data, flags=flags, copy=copy, track=track)
  File "zmq/backend/cython/socket.pyx", line 728, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 775, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 242, in zmq.backend.cython.socket._send_copy
  File "zmq/b

KeyboardInterrupt: 

In [4]:
pd.read_csv("Citymapper_Mobility_Index_20200501.csv")

,Citymapper Mobility Index - Last updated on 2020-05-01,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,Journalists and public-interest researchers ma...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"All others who wish to use this information, p...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date,Amsterdam,Barcelona,Berlin,Birmingham,Boston,Brussels,Chicago,Copenhagen,Hamburg,...,Singapore,St. Petersburg,Stockholm,Sydney,São Paulo,Tokyo,Toronto,Vancouver,Vienna,Washington DC
3,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2020-04-26,0.07,0.03,0.15,0.13,0.05,0.08,0.08,0.12,0.16,...,0.12,0.28,0.27,0.17,0.12,0.05,0.16,0.14,0.09,0.04
101,2020-04-27,0.08,0.05,0.17,0.13,0.04,0.08,0.06,0.14,0.19,...,0.11,0.33,0.33,0.15,0.13,0.04,0.15,0.18,0.1,0.05
102,2020-04-28,0.08,0.05,0.17,0.12,0.05,0.06,0.06,0.15,0.16,...,0.12,0.3,0.29,0.14,0.11,0.06,0.17,0.17,0.12,0.04
103,2020-04-29,0.08,0.05,0.18,0.11,0.05,0.07,0.06,0.17,0.18,...,0.11,0.31,0.33,0.15,0.12,0.07,0.14,0.15,0.11,0.05


In [56]:
idxes_file=np.array([[i[0],i[1]] for i in np.array(df3["ColumnAmountNO2"].index)])

x,y=cairo_cords

a=np.where ( 
    (np.abs(idxes_file[:,0]-x)<=geo_dist_factor)
    & 
    (np.abs(idxes_file[:,1]-y)<=geo_dist_factor)
)[0]

#for i in a:
#    print(df3["ColumnAmountNO2"].index[i])


In [54]:
x,y

(30.0444, 31.2357)

In [43]:
c=cairo_cords
a=[]
for idx,i in enumerate(df3["ColumnAmountNO2"].index):
                
                if isWithinRange(i[0],c[0],geo_dist_factor) and \
                isWithinRange(i[1],c[1],geo_dist_factor):
                    
                    no2_amount=df3["ColumnAmountNO2"][i]
                    if not (not no2_amount>0 and  not no2_amount<0): #basically eval if NaN
                        a.append(no2_amount)

In [44]:
len(a)

758391